Data Preparation (FIXED)
## Notebook 1: Load, Standardize, and Validate All Data Sources

**CRITICAL FIXES APPLIED:**
1.  Use TRI "PARENT COMPANY D and B NR" field (not TRIFD or MAILING_NAME)
2.  Parse CRSP with pipe delimiter and correct column names
3.  Proper FACILITY_STATE handling


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import re
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded successfully")
print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Mounted at /content/drive
✓ Libraries loaded successfully
Analysis started: 2025-12-04 12:35:40


## 1. Define Paths and Parameters

In [ ]:
# Base path
BASE_PATH = Path('/content/drive/MyDrive/Paper1_Dataset')

# Data source paths
TRI_PATH = BASE_PATH / 'EPA TRI'
CRSP_PATH = BASE_PATH / 'crspdata'
EDGAR_PATH = BASE_PATH / 'SEC EDGAR'
SHELDUS_PATH = BASE_PATH / 'SHELDUS'

# Output directory
OUTPUT_PATH = Path('/content/drive/MyDrive/Paper1_Dataset/processed')
OUTPUT_PATH.mkdir(exist_ok=True)

# Analysis time window
START_YEAR = 2009
END_YEAR = 2023

print("Path Configuration:")
print(f"  TRI Data: {TRI_PATH.exists()} ✓" if TRI_PATH.exists() else f"  TRI Data: ✗ NOT FOUND")
print(f"  CRSP Data: {CRSP_PATH.exists()} ✓" if CRSP_PATH.exists() else f"  CRSP Data: ✗ NOT FOUND")
print(f"  EDGAR Data: {EDGAR_PATH.exists()} ✓" if EDGAR_PATH.exists() else f"  EDGAR Data: ✗ NOT FOUND")
print(f"  SHELDUS Data: {SHELDUS_PATH.exists()} ✓" if SHELDUS_PATH.exists() else f"  SHELDUS Data: ✗ NOT FOUND")
print(f"\nAnalysis Window: {START_YEAR}-{END_YEAR} ({END_YEAR - START_YEAR + 1} years)")

Path Configuration:
  TRI Data: True ✓
  CRSP Data: True ✓
  EDGAR Data: True ✓
  SHELDUS Data: True ✓

Analysis Window: 2009-2023 (15 years)


## 2. Load and Parse CRSP Data (FIXED)

**FIX:** Parse with pipe delimiter and extract column 13 (COMNAM) for company names

In [ ]:
def parse_crsp_header():
    """
    Parse CRSP header file correctly.

    CRITICAL FIX: Use pipe delimiter and proper column names.
    Column 13 (index 12) contains actual company names.

    File format:
    PERMNO|CUSIP8|CUSIP9|TICKER|PERMCO|COMPNO|ISSUNO|HEXCD|HSICCD|BEGDAT|ENDDAT|SICCD|COMNAM|...
    """
    print("\n" + "="*80)
    print("PARSING CRSP HEADER FILE (FIXED)")
    print("="*80)

    hdr_file = CRSP_PATH / 'sfz_hdr.dat'

    if not hdr_file.exists():
        print(f"✗ ERROR: {hdr_file} not found!")
        return None

    print(f"Reading: {hdr_file.name} ({hdr_file.stat().st_size / 1e6:.2f} MB)")

    # Define CRSP column names based on documentation
    crsp_columns = [
        'PERMNO',      # 0: Permanent company number
        'CUSIP8',      # 1: CUSIP 8-digit
        'CUSIP9',      # 2: CUSIP 9-digit
        'TICKER',      # 3: Stock ticker (THIS was in 'Unnamed: 3'!)
        'PERMCO',      # 4: Permanent company number
        'COMPNO',      # 5: Company number
        'ISSUNO',      # 6: Issue number
        'HEXCD',       # 7: Exchange code
        'HSICCD',      # 8: SIC code
        'BEGDAT',      # 9: Begin date
        'ENDDAT',      # 10: End date
        'SICCD',       # 11: SIC code
        'COMNAM',      # 12: COMPANY NAME (THIS IS WHAT WE NEED!)
        'TICKER_SYM',  # 13: Ticker symbol
        'NAICS',       # 14: NAICS code
        'PRIMEXCH',    # 15: Primary exchange
        'TRDSTAT',     # 16: Trading status
        'SECSTAT',     # 17: Security status
        'PERMCO_LINKTYPE'  # 18: Link type
    ]

    try:
        # Read with PIPE delimiter
        df = pd.read_csv(hdr_file,
                        delimiter='|',
                        encoding='latin-1',
                        names=crsp_columns,
                        low_memory=False)

        print(f"\n✓ Successfully loaded: {len(df):,} records")
        print(f"  Columns: {list(df.columns)}")

        # Verify COMNAM contains actual company names
        print(f"\nVerifying COMNAM field:")
        print(f"  Non-null: {df['COMNAM'].notna().sum():,}")
        print(f"  Unique companies: {df['COMNAM'].nunique():,}")
        print(f"  Average name length: {df['COMNAM'].str.len().mean():.1f} characters")

        print(f"\nSample company names:")
        samples = df['COMNAM'].dropna().head(20).tolist()
        for i, name in enumerate(samples, 1):
            print(f"  {i:2d}. {name}")

        # Compare TICKER vs COMNAM
        print(f"\nTICKER vs COMNAM comparison:")
        comparison = df[['TICKER', 'COMNAM']].head(10)
        print(comparison.to_string(index=False))

        return df

    except Exception as e:
        print(f"\n✗ Error parsing CRSP header: {str(e)}")
        return None

# Execute
crsp_header = parse_crsp_header()


PARSING CRSP HEADER FILE (FIXED)
Reading: sfz_hdr.dat (4.51 MB)

✓ Successfully loaded: 38,872 records
  Columns: ['PERMNO', 'CUSIP8', 'CUSIP9', 'TICKER', 'PERMCO', 'COMPNO', 'ISSUNO', 'HEXCD', 'HSICCD', 'BEGDAT', 'ENDDAT', 'SICCD', 'COMNAM', 'TICKER_SYM', 'NAICS', 'PRIMEXCH', 'TRDSTAT', 'SECSTAT', 'PERMCO_LINKTYPE']

Verifying COMNAM field:
  Non-null: 38,872
  Unique companies: 32,868
  Average name length: 20.6 characters

Sample company names:
   1. OPTIMUM MANUFACTURING INC
   2. GAS NATURAL INC
   3. BANCTRUST FINANCIAL GROUP INC
   4. GREAT COUNTRY BK ASONIA CT
   5. CLOSE OUTS PLUS INC
   6. WESTERN ENERGY RESOURCES INC
   7. A C F INDUSTRIES INC
   8. SHAREDATA INC
   9. GARDENAMERICA CORP
  10. IROQUOIS BANCORP INC
  11. CABOT MEDICAL CORP
  12. A T C GROUP SERVICES INC
  13. D P A C TECHNOLOGIES CORP
  14. ARIZONA APPETITOS STORES INC
  15. A J INDUSTRIES INC
  16. A & M FOOD SERVICES INC
  17. SCOTT TECHNOLOGIES INC
  18. CONCURRENT COMPUTER CORP
  19. DELTA COMPUTEC INC
 

## 3. Load EPA TRI Data (FIXED)

**FIX:** Extract correct parent company field: "72. PARENT COMPANY D and B NR"

In [ ]:
def load_tri_year(year):
    """
    Load TRI data with CORRECT column mapping.

    CRITICAL FIX: TRI file columns are SHIFTED!
    - Column "13. FACILITY COUNTY" actually contains STATE (OH, CA, TX)
    - Column "14. FACILITY STATE" actually contains ZIP codes
    - Need to remap everything
    """
    year_folder = TRI_PATH / f'us_{year}'

    if not year_folder.exists():
        print(f"  ✗ Data not available for {year}")
        return None

    file_1a = year_folder / f'US_1a_{year}.txt'

    if not file_1a.exists():
        print(f"  ✗ US_1a file not found for {year}")
        return None

    try:
        # Read with tab delimiter
        df = pd.read_csv(
            file_1a,
            delimiter='\t',
            encoding='latin-1',
            on_bad_lines='skip',
            engine='python'
        )

        # CORRECTED column mapping (TRI file is shifted!)
        # The column NAMES in the file don't match the actual DATA
        column_mapping = {
            '9. TRIFD': 'TRIFD',
            '2. REPORTING YEAR': 'REPORTING_YEAR',
            '10. FACILITY NAME': 'FACILITY_STREET',      # Actually contains STREET
            '11. FACILITY STREET': 'FACILITY_CITY',      # Actually contains CITY
            '12. FACILITY CITY': 'FACILITY_COUNTY',      # Actually contains COUNTY
            '13. FACILITY COUNTY': 'FACILITY_STATE',     # Actually contains STATE! ✓
            '14. FACILITY STATE': 'FACILITY_ZIP',        # Actually contains ZIP
            '15. FACILITY ZIP CODE': 'UNKNOWN_1',        # Unknown data
            '72. PARENT COMPANY D and B NR': 'PARENT_COMPANY',
            '73. STANDARDIZED PARENT COMPANY NAME': 'PARENT_COMPANY_STD',
            '18. MAILING NAME': 'MAILING_NAME',
            '19. MAILING STREET': 'MAILING_STREET',
            '20. MAILING CITY': 'MAILING_CITY',
            '21. MAILING STATE': 'MAILING_STATE',
        }

        # Find which columns exist and rename
        rename_map = {}
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                rename_map[old_name] = new_name

        if not rename_map:
            print(f"  ✗ No expected columns found in {year}")
            return None

        # Select and rename
        df_subset = df[list(rename_map.keys())].copy()
        df_subset = df_subset.rename(columns=rename_map)

        # Create COMPANY_NAME field
        df_subset['COMPANY_NAME'] = df_subset['PARENT_COMPANY'].fillna(df_subset.get('TRIFD', ''))
        df_subset['DATA_YEAR'] = year

        print(f"  ✓ {year}: {len(df_subset):,} facilities loaded")

        # DEBUG: Show sample for first year
        if year == START_YEAR:
            print(f"      Sample FACILITY_STATE: {df_subset['FACILITY_STATE'].dropna().head(10).tolist()}")
            print(f"      Sample FACILITY_COUNTY: {df_subset['FACILITY_COUNTY'].dropna().head(10).tolist()}")

        return df_subset

    except Exception as e:
        print(f"  ✗ Error loading {year}: {str(e)}")
        return None

def load_all_tri_data():
    """Load TRI data for all years 2009-2023"""
    print("\n" + "="*80)
    print(f"LOADING EPA TRI DATA WITH CORRECTED COLUMN MAPPING ({START_YEAR}-{END_YEAR})")
    print("="*80)

    all_years = []

    for year in range(START_YEAR, END_YEAR + 1):
        df_year = load_tri_year(year)
        if df_year is not None:
            all_years.append(df_year)

    if not all_years:
        print("\n✗ No TRI data loaded successfully!")
        return None

    # Combine all years
    df_all = pd.concat(all_years, ignore_index=True)

    print(f"\n✓ COMBINED TRI DATA:")
    print(f"  Total records: {len(df_all):,}")
    print(f"  Years loaded: {len(all_years)} (missing: {(END_YEAR - START_YEAR + 1) - len(all_years)})")
    print(f"  Unique facilities: {df_all['TRIFD'].nunique():,}")
    print(f"  With parent company: {df_all['PARENT_COMPANY'].notna().sum():,} ({df_all['PARENT_COMPANY'].notna().sum()/len(df_all)*100:.1f}%)")

    # Verify FACILITY_STATE now contains actual states
    print(f"\n  FACILITY_STATE verification:")
    print(f"    Type: {df_all['FACILITY_STATE'].dtype}")
    print(f"    Sample: {df_all['FACILITY_STATE'].dropna().head(20).tolist()}")
    print(f"    Unique values: {df_all['FACILITY_STATE'].nunique()}")

    return df_all

# Execute
tri_data = load_all_tri_data()


LOADING EPA TRI DATA WITH CORRECTED COLUMN MAPPING (2009-2023)
  ✓ 2009: 81,681 facilities loaded
      Sample FACILITY_STATE: ['OH', 'OH', 'OH', 'OH', 'OH', 'AL', 'AL', 'PA', 'PA', 'MI']
      Sample FACILITY_COUNTY: ['STARK', 'STARK', 'STARK', 'STARK', 'STARK', 'LAUDERDALE', 'LAUDERDALE', 'NORTHAMPTON', 'NORTHAMPTON', 'ALLEGAN']
  ✓ 2010: 82,134 facilities loaded
  ✗ Data not available for 2011
  ✓ 2012: 82,846 facilities loaded
  ✓ 2013: 83,357 facilities loaded
  ✓ 2014: 83,590 facilities loaded
  ✓ 2015: 82,869 facilities loaded
  ✓ 2016: 81,608 facilities loaded
  ✓ 2017: 81,754 facilities loaded
  ✓ 2018: 81,756 facilities loaded
  ✓ 2019: 80,820 facilities loaded
  ✓ 2020: 78,631 facilities loaded
  ✓ 2021: 78,212 facilities loaded
  ✓ 2022: 80,002 facilities loaded
  ✓ 2023: 78,589 facilities loaded

✓ COMBINED TRI DATA:
  Total records: 1,137,849
  Years loaded: 14 (missing: 1)
  Unique facilities: 29,176
  With parent company: 808,030 (71.0%)

  FACILITY_STATE verification:

In [ ]:
# Display sample to verify correct fields
if tri_data is not None:
    print("\nSample TRI Records (showing parent company):")
    print("="*80)
    sample_cols = ['TRIFD', 'PARENT_COMPANY', 'COMPANY_NAME', 'FACILITY_NAME', 'FACILITY_STATE']
    available = [col for col in sample_cols if col in tri_data.columns]
    print(tri_data[available].head(20).to_string(index=False))

    print("\n" + "="*80)
    print("Field Comparison:")
    print("="*80)
    comparison = tri_data[['TRIFD', 'PARENT_COMPANY', 'MAILING_NAME']].drop_duplicates().head(15)
    print(comparison.to_string(index=False))


Sample TRI Records (showing parent company):
                                                TRIFD                    PARENT_COMPANY                                          COMPANY_NAME FACILITY_STATE
                             ALLIANCE CASTINGS CO LLC              OHIO CASTINGS CO LLC                                  OHIO CASTINGS CO LLC             OH
                             ALLIANCE CASTINGS CO LLC              OHIO CASTINGS CO LLC                                  OHIO CASTINGS CO LLC             OH
                             ALLIANCE CASTINGS CO LLC              OHIO CASTINGS CO LLC                                  OHIO CASTINGS CO LLC             OH
                             ALLIANCE CASTINGS CO LLC              OHIO CASTINGS CO LLC                                  OHIO CASTINGS CO LLC             OH
                             ALLIANCE CASTINGS CO LLC              OHIO CASTINGS CO LLC                                  OHIO CASTINGS CO LLC             OH
            

## 4. Standardize Company Names

In [ ]:
def standardize_company_name(name):
    """
    Standardize company name for matching.
    """
    if pd.isna(name) or name == '':
        return ''

    # Convert to string and uppercase
    name = str(name).upper().strip()

    # Replace common separators
    name = re.sub(r'[,\-/]', ' ', name)

    # Replace & with AND
    name = re.sub(r'\s+&\s+', ' AND ', name)

    # Remove legal suffixes
    legal_suffixes = [
        r'\bINCORPORATED\b', r'\bINC\.?\b', r'\bCORPORATION\b', r'\bCORP\.?\b',
        r'\bCOMPANY\b', r'\bCO\.?\b', r'\bLIMITED\b', r'\bLTD\.?\b',
        r'\bLLC\b', r'\bL\.?L\.?C\.?\b', r'\bLP\b', r'\bLLP\b', r'\bPLC\b',
    ]

    for suffix in legal_suffixes:
        name = re.sub(suffix, '', name)

    # Remove punctuation
    name = re.sub(r'[^A-Z0-9\s]', '', name)

    # Remove extra whitespace
    name = re.sub(r'\s+', ' ', name).strip()

    return name

# Test
test_names = [
    'TYSON FOODS INC',
    'General Motors Corporation',
    'Apple Inc.',
    'Johnson & Johnson',
]

print("\nStandardization Test:")
print("="*80)
for name in test_names:
    print(f"{name:40s} → {standardize_company_name(name)}")


Standardization Test:
TYSON FOODS INC                          → TYSON FOODS
General Motors Corporation               → GENERAL MOTORS
Apple Inc.                               → APPLE
Johnson & Johnson                        → JOHNSON AND JOHNSON


In [ ]:
# Apply standardization to TRI
if tri_data is not None:
    print("\nStandardizing TRI company names...")
    tri_data['COMPANY_NAME_STD'] = tri_data['COMPANY_NAME'].apply(standardize_company_name)

    print(f"✓ Complete")
    print(f"  Unique standardized names: {tri_data['COMPANY_NAME_STD'].nunique():,}")

    print("\nSample standardizations:")
    sample = tri_data[['COMPANY_NAME', 'COMPANY_NAME_STD']].drop_duplicates().head(20)
    print(sample.to_string(index=False))

# Apply standardization to CRSP
if crsp_header is not None:
    print("\nStandardizing CRSP company names...")
    crsp_header['COMNAM_STD'] = crsp_header['COMNAM'].apply(standardize_company_name)

    print(f"✓ Complete")
    print(f"  Unique standardized names: {crsp_header['COMNAM_STD'].nunique():,}")

    print("\nSample standardizations:")
    sample = crsp_header[['COMNAM', 'COMNAM_STD']].drop_duplicates().head(20)
    print(sample.to_string(index=False))


Standardizing TRI company names...
✓ Complete
  Unique standardized names: 15,834

Sample standardizations:
                                         COMPANY_NAME                                 COMPANY_NAME_STD
                                 OHIO CASTINGS CO LLC                                    OHIO CASTINGS
                    SOUTHEASTERN EXTRUSION & TOOL INC                  SOUTHEASTERN EXTRUSION AND TOOL
                             POLYTEK DEVELOPMENT CORP                              POLYTEK DEVELOPMENT
                           ALLEGAN METAL FINISHING CO                          ALLEGAN METAL FINISHING
                                            BASF CORP                                             BASF
                                    TITAN AMERICA LLC                                    TITAN AMERICA
CMC STEEL FABRICATORS INC. DBA CMC CAPITOL CITY STEEL CMC STEEL FABRICATORS DBA CMC CAPITOL CITY STEEL
                 SURTECO N.A. INC. JEANNETTE PA PLANT              

## 5. Fix FACILITY_STATE

In [ ]:
print("\n" + "="*80)
print("STEP 5: CREATING STATE_ABBR FROM FACILITY_STATE")
print("="*80)

if tri_data is not None and 'FACILITY_STATE' in tri_data.columns:

    print(f"\nFACILITY_STATE is already in text format (2-letter codes)")
    print(f"  Sample: {tri_data['FACILITY_STATE'].dropna().head(10).tolist()}")

    # Simply clean and copy
    tri_data['STATE_ABBR'] = tri_data['FACILITY_STATE'].str.upper().str.strip()

    converted = tri_data['STATE_ABBR'].notna().sum()

    print(f"\n✓ STATE_ABBR created:")
    print(f"  With STATE_ABBR: {converted:,} ({converted/len(tri_data)*100:.1f}%)")

    if converted > 0:
        print(f"\n  Top 10 states by facility count:")
        print(tri_data['STATE_ABBR'].value_counts().head(10))

else:
    print("\n    Cannot convert: tri_data is None or FACILITY_STATE missing")

print("\n" + "="*80)


STEP 5: CREATING STATE_ABBR FROM FACILITY_STATE

FACILITY_STATE is already in text format (2-letter codes)
  Sample: ['OH', 'OH', 'OH', 'OH', 'OH', 'AL', 'AL', 'PA', 'PA', 'MI']

✓ STATE_ABBR created:
  With STATE_ABBR: 1,141,457 (100.0%)

  Top 10 states by facility count:
STATE_ABBR
TX    123317
OH     73243
IL     54780
PA     53937
CA     53307
IN     48265
MI     42769
LA     41391
WI     39793
NC     32782
Name: count, dtype: int64



## 5.5  CREATE FIPS COUNTY CODES

**CRITICAL ADDITION:** This step creates FIPS county codes

This fixes the downstream disaster linkage issue that was causing 0% exposure.

In [ ]:
print("\n" + "="*80)
print("STEP 5.5: CREATING FIPS COUNTY CODES (NEW!)")
print("="*80)

if tri_data is not None:
    print("\n5.5.1 Downloading Census county FIPS crosswalk...")

    import urllib.request

    url = "https://www2.census.gov/geo/docs/reference/codes/files/national_county.txt"

    try:
        county_data = pd.read_csv(
            url,
            encoding='latin-1',
            names=['STATE_ABBR', 'STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_NAME', 'CLASS']
        )

        # Create 5-digit FIPS code
        county_data['FIPS'] = (
            county_data['STATE_FIPS'].astype(str).str.zfill(2) +
            county_data['COUNTY_FIPS'].astype(str).str.zfill(3)
        )

        print(f"  ✓ Downloaded {len(county_data):,} counties from Census")

    except Exception as e:
        print(f"  ⚠️  Download failed: {e}")
        county_data = None

    if county_data is not None:
        print("\n5.5.2 Standardizing county names with improved algorithm...")

        def standardize_county(name):
            """
            Aggressive county name standardization for maximum match rate.

            Handles common issues:
            - ST. LOUIS → SAINTLOUIS
            - LA SALLE → LASALLE
            - DE KALB → DEKALB
            - MC LEAN → MCLEAN
            """
            if pd.isna(name) or name == '':
                return ''

            name = str(name).upper().strip()

            # Fix common abbreviations
            replacements = {
                'ST.': 'SAINT', 'ST ': 'SAINT',
                'STE.': 'SAINTE', 'STE ': 'SAINTE',
                'MT.': 'MOUNT', 'MT ': 'MOUNT',
                'FT.': 'FORT', 'FT ': 'FORT',
            }
            for old, new in replacements.items():
                name = name.replace(old, new)

            # Remove ALL spaces, periods, hyphens, apostrophes
            for char in [' ', '.', '-', "'", ',']:
                name = name.replace(char, '')

            # Remove common suffixes
            suffixes = ['COUNTY', 'PARISH', 'BOROUGH', 'CITY', 'MUNICIPIO', 'CNTY']
            for suffix in suffixes:
                if name.endswith(suffix):
                    name = name[:-len(suffix)]

            return name.strip()

        # Apply to both datasets
        print("  Standardizing TRI county names...")
        tri_data['FACILITY_COUNTY_STD'] = tri_data['FACILITY_COUNTY'].apply(standardize_county)

        print("  Standardizing Census county names...")
        county_data['COUNTY_NAME_STD'] = county_data['COUNTY_NAME'].apply(standardize_county)

        # Show examples
        print("\n  Standardization examples:")
        examples = [
            'ST. LOUIS', 'SAINT LOUIS', 'LA SALLE', 'DE KALB',
            'MC LEAN', 'PRINCE GEORGES'
        ]
        for ex in examples[:6]:
            print(f"    {ex:20s} → {standardize_county(ex)}")

        print("\n5.5.3 Merging TRI facilities with FIPS codes...")

        # CRITICAL: Check if STATE_ABBR exists
        if 'STATE_ABBR' not in tri_data.columns:
            print(f"  ⚠️  ERROR: STATE_ABBR column not found in tri_data!")
            print(f"     Available columns: {list(tri_data.columns[:10])}...")
            print(f"     FIPS matching cannot proceed.")
            print(f"     Make sure Cell 13 (STATE conversion) ran successfully.")
        else:
            print(f"  ✓ STATE_ABBR found: {tri_data['STATE_ABBR'].notna().sum():,} records")

        # Count before merge
        before_count = len(tri_data)

        # Remove old FIPS if exists
        if 'FIPS' in tri_data.columns:
            tri_data = tri_data.drop(columns=['FIPS'])

        # Merge on standardized names
        tri_data = tri_data.merge(
            county_data[['STATE_ABBR', 'COUNTY_NAME_STD', 'FIPS', 'COUNTY_NAME']],
            left_on=['STATE_ABBR', 'FACILITY_COUNTY_STD'],
            right_on=['STATE_ABBR', 'COUNTY_NAME_STD'],
            how='left',
            suffixes=('', '_census')
        )

        # Statistics
        after_count = len(tri_data)
        matched_count = tri_data['FIPS'].notna().sum()
        match_rate = matched_count / after_count * 100

        print(f"\n✓ FIPS MATCHING COMPLETE:")
        print(f"  Rows before: {before_count:,}")
        print(f"  Rows after: {after_count:,}")
        print(f"  With FIPS: {matched_count:,}")
        print(f"  Match rate: {match_rate:.1f}%")

        # Quality assessment
        if match_rate >= 70:
            print(f"\n  ✓ EXCELLENT: {match_rate:.1f}% match rate!")
        elif match_rate >= 50:
            print(f"\n  ✓ GOOD: {match_rate:.1f}% match rate")
        else:
            print(f"\n  ⚠️  LOW: {match_rate:.1f}% match rate")
            print(f"  Investigating unmatched counties...")

            unmatched = tri_data[tri_data['FIPS'].isna()]
            if len(unmatched) > 0:
                top_unmatched = unmatched.groupby(
                    ['STATE_ABBR', 'FACILITY_COUNTY']
                ).size().sort_values(ascending=False).head(15)

                print(f"\n  Top 15 unmatched counties:")
                print(f"  {'State':<6} {'County':<35} {'Clean':<20} {'Count':<8}")
                print(f"  {'-'*75}")
                for (state, county), count in top_unmatched.items():
                    clean = standardize_county(county)
                    print(f"  {state:<6} {county:<35} {clean:<20} {count:>6,}")

        # Show matched samples
        print(f"\n  Sample matched facilities:")
        matched_sample = tri_data[tri_data['FIPS'].notna()][['STATE_ABBR', 'FACILITY_COUNTY', 'FIPS']].drop_duplicates().head(10)
        print(matched_sample.to_string(index=False))

        # Save crosswalk for other notebooks
        print("\n5.5.4 Saving county crosswalk for downstream notebooks...")
        crosswalk_file = OUTPUT_PATH / 'county_fips_crosswalk.csv'
        county_data[['STATE_ABBR', 'COUNTY_NAME', 'FIPS']].to_csv(crosswalk_file, index=False)
        print(f"  ✓ Saved: {crosswalk_file}")

    else:
        print("\n⚠️  Could not create FIPS codes - crosswalk unavailable")
        tri_data['FIPS'] = None

else:
    print("\n⚠️  tri_data is None - cannot create FIPS codes")

print("\n" + "="*80)


STEP 5.5: CREATING FIPS COUNTY CODES (NEW!)

5.5.1 Downloading Census county FIPS crosswalk...
  ✓ Downloaded 3,235 counties from Census

5.5.2 Standardizing county names with improved algorithm...
  Standardizing TRI county names...
  Standardizing Census county names...

  Standardization examples:
    ST. LOUIS            → SAINTLOUIS
    SAINT LOUIS          → SAINTLOUIS
    LA SALLE             → LASALLE
    DE KALB              → DEKALB
    MC LEAN              → MCLEAN
    PRINCE GEORGES       → PRINCEGEORGES

5.5.3 Merging TRI facilities with FIPS codes...
  ✓ STATE_ABBR found: 1,141,457 records

✓ FIPS MATCHING COMPLETE:
  Rows before: 1,141,457
  Rows after: 1,148,673
  With FIPS: 1,136,098
  Match rate: 98.9%

  ✓ EXCELLENT: 98.9% match rate!

  Sample matched facilities:
STATE_ABBR         FACILITY_COUNTY  FIPS
        OH                   STARK 39151
        AL              LAUDERDALE 01077
        PA             NORTHAMPTON 42095
        MI                 ALLEGAN 26005


## 6. Save Processed Data

In [ ]:
print("\n" + "="*80)
print("SAVING PROCESSED DATA")
print("="*80)

# Save TRI
if tri_data is not None:
    output_file = OUTPUT_PATH / 'tri_facilities_panel.parquet'
    tri_data.to_parquet(output_file, index=False)
    print(f"✓ TRI saved: {output_file}")
    print(f"  Records: {len(tri_data):,}")
    print(f"  Columns: {list(tri_data.columns)}")

# Save CRSP
if crsp_header is not None:
    output_file = OUTPUT_PATH / 'crsp_companies.parquet'
    crsp_header.to_parquet(output_file, index=False)
    print(f"\n✓ CRSP saved: {output_file}")
    print(f"  Records: {len(crsp_header):,}")
    print(f"  Columns: {list(crsp_header.columns)[:10]}...")

print("\n" + "="*80)
print("DATA PREPARATION COMPLETE!")
print("="*80)


SAVING PROCESSED DATA
✓ TRI saved: /content/drive/MyDrive/Paper1_Dataset/processed/tri_facilities_panel.parquet
  Records: 1,148,673
  Columns: ['TRIFD', 'REPORTING_YEAR', 'FACILITY_STREET', 'FACILITY_CITY', 'FACILITY_COUNTY', 'FACILITY_STATE', 'FACILITY_ZIP', 'UNKNOWN_1', 'PARENT_COMPANY', 'PARENT_COMPANY_STD', 'MAILING_NAME', 'MAILING_STREET', 'MAILING_CITY', 'MAILING_STATE', 'COMPANY_NAME', 'DATA_YEAR', 'COMPANY_NAME_STD', 'STATE_ABBR', 'FACILITY_COUNTY_STD', 'COUNTY_NAME_STD', 'COUNTY_NAME', 'COUNTY_NAME_STD_census', 'FIPS', 'COUNTY_NAME_census']

✓ CRSP saved: /content/drive/MyDrive/Paper1_Dataset/processed/crsp_companies.parquet
  Records: 38,872
  Columns: ['PERMNO', 'CUSIP8', 'CUSIP9', 'TICKER', 'PERMCO', 'COMPNO', 'ISSUNO', 'HEXCD', 'HSICCD', 'BEGDAT']...

DATA PREPARATION COMPLETE!

✅ Ready for Notebook 2: Automated Matching

Expected improvements:
  - Match rate should increase from 0.2% to 15-25%
  - 5,000-8,000 high-quality matches expected
  - Parent company coverage: ~6

In [ ]:
# DIAGNOSTIC: Find the real STATE column in TRI data
print("="*80)
print("DIAGNOSTIC: FINDING ACTUAL STATE COLUMN")
print("="*80)

if tri_data is not None:
    print(f"\nAll columns in tri_data:")
    for i, col in enumerate(tri_data.columns):
        print(f"  {i:2d}. {col}")

    print(f"\n\nSample of ALL columns for first facility:")
    print("="*80)
    sample = tri_data.head(1).T
    sample.columns = ['Value']
    print(sample.to_string())

    print(f"\n\nLooking for STATE-like data...")
    print("="*80)

    # Check each column for 2-letter state codes
    for col in tri_data.columns:
        if tri_data[col].dtype == 'object':  # Only check text columns
            sample_vals = tri_data[col].dropna().head(100).unique()
            # Check if values look like state codes (2 chars, all uppercase)
            state_like = [str(v) for v in sample_vals if isinstance(v, str) and len(str(v).strip()) == 2 and str(v).strip().isupper()]
            if len(state_like) > 3:
                print(f"\n✓ Column '{col}' contains state-like values:")
                print(f"  Sample: {state_like[:20]}")
                print(f"  Unique: {len(state_like)}")

print("\n" + "="*80)

DIAGNOSTIC: FINDING ACTUAL STATE COLUMN

All columns in tri_data:
   0. TRIFD
   1. REPORTING_YEAR
   2. FACILITY_STREET
   3. FACILITY_CITY
   4. FACILITY_COUNTY
   5. FACILITY_STATE
   6. FACILITY_ZIP
   7. UNKNOWN_1
   8. PARENT_COMPANY
   9. PARENT_COMPANY_STD
  10. MAILING_NAME
  11. MAILING_STREET
  12. MAILING_CITY
  13. MAILING_STATE
  14. COMPANY_NAME
  15. DATA_YEAR
  16. COMPANY_NAME_STD
  17. STATE_ABBR
  18. FACILITY_COUNTY_STD
  19. COUNTY_NAME_STD
  20. COUNTY_NAME
  21. COUNTY_NAME_STD_census
  22. FIPS
  23. COUNTY_NAME_census


Sample of ALL columns for first facility:
                                           Value
TRIFD                   ALLIANCE CASTINGS CO LLC
REPORTING_YEAR                                NO
FACILITY_STREET               1001 E BROADWAY ST
FACILITY_CITY                           ALLIANCE
FACILITY_COUNTY                            STARK
FACILITY_STATE                                OH
FACILITY_ZIP                               44601
UNKNOWN_1     